<a href="https://colab.research.google.com/github/michel-vbarboza/ERA5-Temperature-Prediction/blob/main/Recurrent_Neural_Network_(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [1]:
import pickle
import pandas as pd
import numpy as np
#import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/MMEAD Dissertacao/Dados/'

# Preprocessing

In [4]:
with open(path+'train.pkl', 'rb') as f:
    train = pickle.load(f)

with open(path+'test.pkl', 'rb') as f:
    test = pickle.load(f)

del f

In [5]:
targets = ['target_1', 'target_3', 'target_5',
           'target_7', 'target_10', 'target_15']

features = train.columns.difference(targets)

In [6]:
target = 'target_1'

In [7]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [8]:
# Ensure data is in numpy array format
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Model

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, Input, LayerNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.mixed_precision import set_global_policy

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable XLA (Accelerated Linear Algebra) compilation
tf.config.optimizer.set_jit(True)

# Set up mixed precision training
set_global_policy('mixed_float16')

# Verify GPU usage and mixed precision
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Mixed precision policy:", tf.keras.mixed_precision.global_policy())

# Define the enhanced LSTM model
model = Sequential()

# Input layer specifying the input shape (timesteps, num_features)
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

# Bidirectional LSTM layer with 100 units
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting

# Second LSTM layer with 50 units
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting

# Dense layer with LayerNormalization
model.add(Dense(1, dtype='float32'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error on Test Data: {rmse}")


TensorFlow version: 2.17.0
Num GPUs Available:  1
Mixed precision policy: <DTypePolicy "mixed_float16">
Epoch 1/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 85s 6ms/step - loss: 37636.8242 - val_loss: 22.8983
Epoch 2/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 78s 6ms/step - loss: 708.4182 - val_loss: 12.3271
Epoch 3/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 78s 5ms/step - loss: 664.9321 - val_loss: 10.6178
Epoch 4/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 72s 5ms/step - loss: 622.3248 - val_loss: 10.2060
Epoch 5/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 590.8652 - val_loss: 9.4494
Epoch 6/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 73s 5ms/step - loss: 554.3229 - val_loss: 7.3859
Epoch 7/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 83s 6ms/step - loss: 519.3023 - val_loss: 9.0974
Epoch 8/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 72s 5ms/step - loss: 486.3623 - val_loss: 9.0269
Epoch 9/100
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 457.0168 - val_loss: 6.9382
Epoch 10/100
13355/13355 ━━━━━━━━━━━━━━━━━━━

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Input, Attention, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import set_global_policy

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable XLA (Accelerated Linear Algebra) compilation
tf.config.optimizer.set_jit(True)

# Set up mixed precision training
set_global_policy('mixed_float16')

# Verify GPU usage and mixed precision
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Mixed precision policy:", tf.keras.mixed_precision.global_policy())

# Define the input
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))

# Bidirectional LSTM with dropout
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(inputs)
x = LSTM(50, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)(x)

# Attention mechanism
attention = Attention()([x, x])

# Reduce the sum of the attention weights across the time steps using Lambda layer
context_vector = Lambda(lambda x: tf.reduce_sum(x, axis=1))(attention)

# Output layer
outputs = Dense(1, dtype='float32')(context_vector)

# Define the model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error on Test Data: {rmse}")


TensorFlow version: 2.17.0
Num GPUs Available:  1
Mixed precision policy: <DTypePolicy "mixed_float16">
Epoch 1/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 179s 13ms/step - loss: 37362.0977 - val_loss: 20.2543
Epoch 2/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 199s 13ms/step - loss: 11.3245 - val_loss: 5.8969
Epoch 3/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 207s 13ms/step - loss: 6.4974 - val_loss: 5.3456
Epoch 4/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 194s 13ms/step - loss: 6.0268 - val_loss: 5.3605
Epoch 5/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 204s 13ms/step - loss: 5.8117 - val_loss: 4.9823
Epoch 6/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 198s 12ms/step - loss: 5.6476 - val_loss: 4.9988
Epoch 7/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 203s 13ms/step - loss: 5.5225 - val_loss: 4.9366
Epoch 8/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 168s 13ms/step - loss: 5.3880 - val_loss: 4.8582
Epoch 9/200
13355/13355 ━━━━━━━━━━━━━━━━━━━━ 203s 13ms/step - loss: 5.3342 - val_loss: 4.9303
Epoch 10/200
13355/13355 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
# Save lists using pickle
with open(path+'pred_lstm_'+target+'.pkl', 'wb') as f:
    pickle.dump(y_pred, f)

with open(path+'rmse_lstm_'+target+'.pkl', 'wb') as f:
    pickle.dump(rmse, f)
